In [ ]:
import pymongo

# create client connection to database
client = pymongo.MongoClient('localhost', 27017)
# client = pymongo.MongoClient('127.0.0.1', 27017)

# create database
database = client['school']

# create collection
collection = database['student']

# insert document
#document = { "_id":1, "name":"Tom", "dob":'2003-01-31' }
#document = { "_id":2, "name":"Mary", "dob":'2003-12-30' }
#x = collection.insert_one(document)
#print(x.inserted_id)

# query documents
results = collection.find({})
print(results)
for result in results:
  print(result)

# close client connection
client.close()

In [ ]:
# NoSQL
# - optional fields
# - different field types

import pymongo
import json

# load / loads
# dump / dumps

# create connection
client = pymongo.MongoClient('localhost', 27017)

# create database
database = client['school']

# create collection
collection = database['student'] # equivalent to table in SQL

# CRUD
# C - create/insert
# R - read/select/query
# U - update/edit
# D - delete/remove

# insert document
document = { "_id":99, "name":"Tom", "dob":"2003-01-31" } # equivalent to record
x = collection.insert_one(document)
print(x.inserted_id)

# query
results = collection.find({})
for result in results:
  print(result)

# insert documents
documents = [ { "_id":35, "name":"Mary", "dob":"2003-09-09" }, \
             { "_id":36, "name":"Ali", "dob":"2003-09-10" } ]
x = collection.insert_many(documents)
print(x.inserted_ids)

# query
results = collection.find({})
for result in results:
  print(result)

# update
old_value = { "dob":"2003-03-05" }
new_value = { "$set" : { "dob":"2003-08-12" } }
collection.update_one(old_value, new_value)
collection.update_many(old_value, new_value)

# query
results = collection.find({})
for result in results:
  print(result)
 
# delete
to_delete = {  } 


# close connection
client.close()

In [ ]:
# programming style
##- meaningful identifier names
##- appropriate comments
##- appropriate white space (blank lines, indentation, spaces)

# import module
import pymongo

# create connection to MongoDB server
# localhost = 127.0.0.1
client = pymongo.MongoClient('localhost', 27017)

# create database
database = client['library'] 


# create collection
collection = database['books']

# insert into collection
##documents = [{"book_id": 234, "title": "Father Night", "author": "Kurt", "publisher": "APress", "page_count": 433, "year": "2018"},
##{"book_id": 134, "title": "Mother Night", "author": ["Kurt", "Dan"], "publisher": "APress", "year": "2015"},
##{"book_id": 334, "title": "Programming C## 6.0", "author": ["Andrew", "Dan"], "page_count": 300, "year": "2000"},
##{"book_id": 534, "title": "Introduction to Python", "publisher": "MPH", "year": "1999"},
##{"book_id": 434, "title": "Travel with Dogs", "author": "Andy", "publisher": "APress", "page_count": 100, "year": "2017"}
##]             
##collection.insert_many(documents)

# query collection
##docs = collection.find()
##for doc in docs:
##  print(doc)

# delete collection
#collection.drop()

# get book_id, title, author for books published in 2015
##criteria = { "year": "2015" }
##docs = collection.find(criteria, { "book_id": 1, "title": 1, "author": 1, "_id": 0 })
##for doc in docs:
##  print(doc)

# 100 <= page_count < 400
#criteria = { "$and": [{"page_count": { "$gte": 100 }}, {"page_count": { "$lt": 400 }}]}
##criteria = { "page_count": {"$gte":100,"$lt":400} }
##docs = collection.find(criteria)
##for doc in docs:
##  print(doc)             

# update page_count for documents without page_count to "Less Than 100 Pages"
##criteria = { "page_count": {"$exists": False} }
##new_value = { "$set": {"page_count": "Less Than 100 Pages"} }
##collection.update_many(criteria, new_value)
##docs = collection.find()
##for doc in docs:
##  print(doc)

# sort book title, page_count, year in descending year order
docs = collection.find({}, {"title":1, "page_count":1, "year":1, "_id":0}).sort("year",-1)
#docs = collection.find({}, {"title":1, "page_count":1, "year":1, "_id":0}).sort("year")
for doc in docs:
  print(doc)

# close connection
client.close()